In [ ]:
%pylab
from scipy.stats import sem
import sfg2d.talk
import json

%load_ext autoreload
%autoreload 2
import sfg2d

Import Data

In [ ]:
record = sfg2d.SfgRecord('')
record.name = ''
record.lname = ''
record.pump_freq = 0
record

# Static Spectrum

In [ ]:
sfg2d.plot_record_static(na2so4, save=True)

# Contour Plot

In [ ]:
sfg2d.plot_record_contour(na2so4, save=True, levels=(84, 110, 15))

# Four Level Fit

In [ ]:
record = 
sl = slice(2550, 2650) # Slice in Wavenumbers

x, y, yerr = record.trace(roi_wavenumber=sl)
x = x.squeeze()
y = y.squeeze()
yerr = yerr.squeeze()
fargs = {
     'limit_c': (-10, 10),
     'limit_mu': (-800, 0),
     'limit_s': (0.0001, 1),
     'limit_t1': (1, 10000),
     'limit_t2': (1, 10000),
     's': 0.015,
     't1': 829.0,
     't2': 700.0,
     'c': 0.1,
     'mu': -500.0,
     'fix_t2': True,
}
record.models['{}-{}'.format(sl.start, sl.stop)] = sfg2d.models.FourLevelMolKinM(
    xdata=x, 
    ydata=y.squeeze(), 
    sigma=yerr.squeeze(),
    fitarg=fargs,
    rtol=2.09e-13,
    atol=3.00e-20,
    gSigma=264,
    roi=slice(3, None)

)
record.models

## Run the Fits

In [ ]:
for name, model in record.models.items():
    print(name)
    stats, params = model.minuit.migrad()
    with open("cache/{}_trace_pump{}_{}.json".format(record.name, name, record.pump_freq), "w") as file:
        json.dump(model.minuit.fitarg, file, indent=2)

## Plot Fit Results

In [ ]:
#record.models['2550-2650'].box_coords = (1600, .900)

for name, model in record.models.items():
    sfg2d.plot_model_trace(name, model, record, ylim=(.87, 1.08), xlim=(-1100, 5000), save=True)